In [1]:
# Import necessary modules:
# os: to access environment variables (like HF_TOKEN)
# json: to work with JSON data (e.g., converting Python objects to JSON strings)
# InferenceClient from huggingface_hub: to interact with Hugging Face models via API

import os
import json
from huggingface_hub import InferenceClient

In [2]:
# Open the file "User_Messages.txt" in read mode with UTF-8 encoding
# and read its entire content into the variable `chat_messages`.

with open("User_Messages.txt", "r", encoding="utf-8") as f:
    chat_messages = f.read()

In [3]:
# Initialize the Hugging Face InferenceClient with the API key.

client = InferenceClient(
    api_key=os.environ["HF_TOKEN"],
)

In [4]:
# Helper function to call the model

def run_llm(prompt: str):
    response = client.chat.completions.create(
        model="Qwen/Qwen2.5-7B-Instruct",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message["content"]


In [5]:
# Defining a prompt for a memory extraction engine.

memory_extraction_prompt = f"""
You are a memory extraction engine. 
Given the following 30 chat messages:

--- BEGIN MESSAGES ---
{chat_messages}
--- END MESSAGES ---

Extract structured JSON with:
- user_preferences: list of things the user likes, prefers, or repeatedly asks for  
- emotional_patterns: recurring emotions, tone, frustration, excitement, etc.
- facts_to_remember: long-term facts about the user that matter for personalization  

Output ONLY valid JSON. No explanation.
"""


In [6]:
# Send the memory extraction prompt to the LLM and get the output as a string.

memory_json_str = run_llm(memory_extraction_prompt)

In [7]:
memory_json_str

'```json\n{\n  "user_preferences": [\n    "short explanations",\n    "friendly answers",\n    "simple analogies",\n    "practical answers",\n    "step-by-step learning",\n    "small projects",\n    "building AI apps",\n    "practical answers",\n    "direct answers",\n    "Python over Java",\n    "small AI apps",\n    "LangChain projects",\n    "AI chatbot projects"\n  ],\n  "emotional_patterns": [\n    "confused",\n    "overwhelmed",\n    "excited",\n    "scared",\n    "stressed",\n    "motivated",\n    "lost",\n    "worrying"\n  ],\n  "facts_to_remember": [\n    "studying ECE but interested in AI",\n    "applies to AI master\'s programs in Germany",\n    "practices PySpark",\n    "wants to shift into AI",\n    "wants to study in Germany for AI research",\n    "tries to balance RRB JE prep and AI learning",\n    "wants to improve profile without experience",\n    "applies for government jobs and private AI jobs"\n  ]\n}\n```'

In [8]:
# Attempt to parse the JSON string returned by the LLM (`memory_json_str`) into a Python dictionary.

try:
    memory = json.loads(memory_json_str)
except:
    memory = {"error": "LLM returned invalid JSON", "raw": memory_json_str}


In [9]:
# Define different personality styles for the assistant.
# Each key represents a style, and the value describes how the assistant should communicate.

personality_styles = {
    "mentor": "Speak slowly, clearly, positively. Give thoughtful, wise guidance.",
    "friend": "Be playful, humorous, casual, and fun while staying helpful.",
}

In [10]:
# Define a function to create a personalized prompt for the AI assistant.

def personality_prompt(user_input, persona_name, persona_style, memory):
    return f"""
You are an AI assistant with the personality of a {persona_name}.  
Personality Style: {persona_style}

You have access to user memory:
{json.dumps(memory, indent=2)}

Respond to the user's message below:

USER MESSAGE: "{user_input}"

Give a short, friendly answer.
"""

In [11]:
# Generate a personalized response from the AI assistant based on the user's input.

def generate_personalized_response(user_input, persona="friend"):
    persona_style = personality_styles[persona]
    prompt = personality_prompt(user_input, persona, persona_style, memory)
    return run_llm(prompt)


In [12]:
#User's prompt
user_test_message = "Can you tell me how I improve myself to be calm?"

In [13]:
# Compare the AI assistant's responses before and after applying personality.

before_prompt = f"""
User message: "{user_test_message}"
Give a neutral assistant reply. No personality.
"""
before_reply = run_llm(before_prompt)

# After personality
after_reply = generate_personalized_response(
    user_test_message, 
    persona="friend"
)

print("\n=== BEFORE")
print(before_reply)

print("\n=== AFTER")
print(after_reply)



=== BEFORE
Improving oneself to be more calm can be achieved through various methods. Regular practice of mindfulness and meditation can help in managing stress and enhancing focus. Engaging in regular physical activity can also contribute to emotional stability. Establishing a consistent sleep schedule and maintaining a healthy diet are also beneficial. Additionally, setting realistic goals and learning to manage time effectively can reduce feelings of overwhelm.

=== AFTER
Of course! Imagine your mind is like a garden, and calmness is like a gentle breeze. To cultivate this breeze, try these little steps:

1. **Breathe Deeply**: Just like how a plant needs water, your mind needs oxygen. Take a few deep breaths whenever you feel overwhelmed. It’s like giving your mind a quick drink of water.

2. **Mindfulness**: Think of it as planting a seed in your mind. Spend a few minutes each day focusing on the present moment. It’s like watching your garden grow without worrying about what’s ne

In [14]:
user_test_message = "Can you tell me how I can overcome my stress?"

In [15]:
before_prompt = f"""
User message: "{user_test_message}"
Give a neutral assistant reply. No personality.
"""
before_reply = run_llm(before_prompt)

# After personality (e.g., witty friend)
after_reply = generate_personalized_response(
    user_test_message, 
    persona="friend"
)

print("\n=== BEFORE")
print(before_reply)

print("\n=== AFTER")
print(after_reply)



=== BEFORE
To overcome stress, consider implementing strategies such as regular physical activity, maintaining a balanced diet, ensuring adequate sleep, practicing relaxation techniques like deep breathing or meditation, and setting realistic goals and priorities. Additionally, seeking support from friends, family, or a professional can also be beneficial.

=== AFTER
Of course! Stress can be a real buzzkill, but don't worry, you've got this! First, let's break it down into manageable steps. You can try:

1. **Deep Breathing:** Just like magic, deep breaths can work wonders. Try inhaling slowly through your nose, holding for a few seconds, and then exhaling through your mouth. Repeat a few times.

2. **Quick Breaks:** When you feel overwhelmed, take a quick break. Stand up, stretch, or walk around for a bit. It helps clear your mind.

3. **Mindfulness:** Spend a few minutes each day focusing on the present moment. It could be as simple as observing your breath or listening to calming m